In [ ]:
### IMPORTS
# TensorFlow
# import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
# NLTK
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
# Sci-kit Learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
# Outros Auxiliares
import numpy as np
import pandas as pd
import re

nltk.download('stopwords')

## inicializando dataset e dividindo entre dados e rotulos
df = pd.read_csv("fake_train.csv")
print("og",df.shape)
df = df.dropna()
X = df.drop('label',axis=1)
y = df['label']
print(X.shape, y.shape)
# df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


og (20800, 5)
(18285, 4) (18285,)


In [ ]:
### PRÉ-PROCESSAMENTO
# resetando index
docs = X.copy()
docs.reset_index(inplace=True)

ps = PorterStemmer()
corpus=[]

# removendo pontuação e stopwords, fazendo stemming
for i in range(0,len(docs)):
  doc = re.sub('[^a-zA-Z]', ' ', docs['title'][i])
  doc = doc.lower()
  doc = doc.split()
  doc = [ps.stem(word) for word in doc if (not word in stopwords.words('english'))]
  doc = ' '.join(doc)
  corpus.append(doc)
# print(corpus)

# onehot encoder
vocab_size = 18000
onehot_repr = [one_hot(words, vocab_size) for words in corpus]

# formatando pra embedding
max_len = 20
embeded_docs = pad_sequences(onehot_repr, maxlen = max_len)

In [ ]:
### MODELO
model = Sequential()
model.add(Embedding(vocab_size, 40, input_length=max_len))
model.add(LSTM(100, activation="relu"))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            720000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 776501 (2.96 MB)
Trainable params: 776501 (2.96 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
X_final = np.array(embeded_docs)
y_final = np.array(y)
print(X_final.shape,y_final.shape)
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.3,random_state=42)
# treinamento
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)
y_pred = (model.predict(X_test) > 0.5)
y_pred
print("\nAccuracy: ",accuracy_score(y_test,y_pred)),
confusion_matrix(y_test,y_pred)

(18285, 20) (18285,)
Epoch 1/10
200/200 [==============================] - 10s 41ms/step - loss: 0.1040 - accuracy: 0.8415 - val_loss: 0.0579 - val_accuracy: 0.9213
Epoch 2/10
200/200 [==============================] - 10s 49ms/step - loss: 0.0368 - accuracy: 0.9519 - val_loss: 0.0567 - val_accuracy: 0.9244
Epoch 3/10
200/200 [==============================] - 11s 54ms/step - loss: 0.0220 - accuracy: 0.9735 - val_loss: 0.0615 - val_accuracy: 0.9240
Epoch 4/10
200/200 [==============================] - 8s 41ms/step - loss: 0.0137 - accuracy: 0.9839 - val_loss: 0.0610 - val_accuracy: 0.9240
Epoch 5/10
200/200 [==============================] - 10s 52ms/step - loss: 0.0103 - accuracy: 0.9884 - val_loss: 0.0666 - val_accuracy: 0.9207
Epoch 6/10
200/200 [==============================] - 10s 49ms/step - loss: 0.0079 - accuracy: 0.9916 - val_loss: 0.0652 - val_accuracy: 0.9213
Epoch 7/10
200/200 [==============================] - 8s 39ms/step - loss: 0.0060 - accuracy: 0.9935 - val_loss: 0.0

array([[2840,  267],
       [ 157, 2222]])

lstm é um tipo de rnn

o problema da rnn é toda a memoria é long-term e ao longo do tempo isso vai se acumulando a ponto q a info nova perde valor

célula state da lstm > 3 gates

3 gates indo de 0-1: forget (regula o que deve ser esquecido), input (oq deve ser adicionado/updatado) e output (o que deve influenciar no output desse passo específico)